In [1]:
import logging

import pandas as pd
from sqlalchemy import select

from sql.models import Album, Artist, Session, Track, engine

In [2]:
session = Session()

In [3]:
britney = Artist(name="Britney Spears")
radar = Track(title="Radar", duration=229, artist=britney)
circus = Track(title="Circus", duration=192, artist=britney)
Circus = Album(name="Circus", year=2008, tracks=[radar, circus])

In [4]:
session.add_all([radar, circus, Circus, britney])
session.commit()

In [5]:
taylor = Artist(name="Taylor Swift")
taylor.albums = [
    Album(name="Red", year=2012, tracks=[
        Track(title="Red", duration=223, artist=taylor),
        Track(title="Starlight", duration=220, artist=taylor),
    ])
]

In [6]:
session.add(taylor)
session.commit()

In [7]:
statement = select(Track).join(Artist).filter(Artist.name == "Britney Spears")
britney_tracks = (
    session.scalars(statement).all()
)
britney_tracks

[<Track: "Radar">, <Track: "Circus">]

In [8]:
statement = select(Artist).filter_by(name="Taylor Swift")
taylor = session.scalar(statement)
taylor

<Artist: Taylor Swift>

In [9]:
taylor.tracks

[<Track: "Red">, <Track: "Starlight">]

In [10]:
query = """
SELECT * FROM tracks
JOIN artists ON tracks.artist_id == artists.id
WHERE artists.name LIKE '%%Britney%%'
"""

In [11]:
df = pd.read_sql(query, con=engine)
df

,id,title,duration,artist_id,album_id,name
0,1,Radar,229,1,1,Britney Spears
1,1,Circus,192,1,1,Britney Spears


In [12]:
session.close()